In [517]:
import numpy as np
import pandas as pd
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')

In [424]:
path = '/Users/yiliwang/Documents/Berkeley MFE/AFP/news data/'
csvfiles = []
for filename in os.listdir(path):
    if filename[-4:] == ".csv":
        csvfiles.append(filename)

In [426]:
ticker_list = ['BAC', 'CHK', 'D', 'GE', 'WMT', 'F', 'TWTR']

In [581]:
def news_organizer(ticker_list, filenames):
    data = pd.read_csv(path+filenames[0])
    data['TimeOfArrival'] = [datetime.strptime(t, '%Y-%m-%d %H:%M:%S') for t in [i[:-4] for i in data['TimeOfArrival']]]
    data = data.set_index('TimeOfArrival')
    data['arr_time'] = data.index
    _intervals_ = data.groupby(pd.TimeGrouper('5Min'))['Ticker'].count().keys()
    _intervals_ = [str(itv)[-8:] for itv in _intervals_]
    df_ini = pd.DataFrame({'_time_':_intervals_})
    
    freq_news_collection = {}
    for ticker in ticker_list:
        _ticker_ = data[data['Ticker'] == ticker]
        _df_ = pd.DataFrame({
            'time_bin':_ticker_.groupby(pd.TimeGrouper('5Min'))['Ticker'].count().keys(),
            'count_news':list(_ticker_.groupby(pd.TimeGrouper('5Min'))['Ticker'].count())
        })
        _df_ = _df_.join(df_ini, how='outer')
        freq_news_collection[ticker] = _df_[['_time_']]
    
    for filename in filenames:
        data = pd.read_csv(path+filename)
        data['TimeOfArrival'] = [datetime.strptime(t, '%Y-%m-%d %H:%M:%S') for t in [i[:-4] for i in data['TimeOfArrival']]]
        data = data.set_index('TimeOfArrival')
        data['arr_time'] = data.index
        
        for ticker in ticker_list:
            _ticker_ = data[data['Ticker'] == ticker]
            _df_ = pd.DataFrame({
            'time_bin':_ticker_.groupby(pd.TimeGrouper('5Min'))['Ticker'].count().keys(),
            'count_news':list(_ticker_.groupby(pd.TimeGrouper('5Min'))['Ticker'].count())
        })
            _df_.columns = ['time '+ticker, filename[:-4]]
            _df_ = _df_[[filename[:-4]]].join(freq_news_collection[ticker], how='outer')#[['count_news']]
            freq_news_collection[ticker] = _df_.fillna(0)
    return freq_news_collection

In [590]:
news_count = news_organizer(ticker_list, csvfiles) # up to 5mins to run

In [591]:
# modify the entry type to generate neat dictionary of dataframes
def dic_modifier(dic, ticker_list):
    _dic_ = {}
    for ticker in ticker_list:
        df_by_ticker = dic[ticker]
        df_by_ticker = df_by_ticker.reset_index(drop=True)
        df_by_ticker = df_by_ticker.set_index('_time_')
        df_by_ticker.columns = [datetime.strptime(dt, '%m_%d_%Y') for dt in df_by_ticker.columns]
        _dic_[ticker] = df_by_ticker.transpose().sort_index()
    return _dic_

In [592]:
# output
dic_modifier(news_count, ticker_list)['BAC']

_time_,10:30:00,10:35:00,10:40:00,10:45:00,10:50:00,10:55:00,11:00:00,11:05:00,11:10:00,11:15:00,...,14:15:00,14:20:00,14:25:00,14:30:00,14:35:00,14:40:00,14:45:00,14:50:00,14:55:00,15:00:00
2016-01-05,4.0,2.0,0.0,2.0,0.0,0.0,2.0,1.0,0.0,5.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
2016-01-06,2.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,6.0,6.0,...,1.0,1.0,3.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0
2016-01-07,3.0,2.0,7.0,5.0,2.0,2.0,3.0,1.0,0.0,1.0,...,2.0,1.0,4.0,2.0,0.0,3.0,1.0,0.0,0.0,2.0
2016-01-08,1.0,0.0,0.0,5.0,3.0,1.0,3.0,0.0,2.0,3.0,...,2.0,2.0,0.0,1.0,2.0,4.0,3.0,6.0,4.0,1.0
2016-01-09,3.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2016-01-10,2.0,1.0,3.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-11,3.0,8.0,4.0,6.0,10.0,9.0,1.0,3.0,2.0,2.0,...,2.0,6.0,4.0,1.0,1.0,0.0,0.0,7.0,2.0,0.0
2016-01-12,5.0,2.0,4.0,3.0,2.0,2.0,1.0,2.0,3.0,4.0,...,2.0,2.0,4.0,1.0,0.0,1.0,1.0,0.0,3.0,0.0
2016-01-13,1.0,0.0,2.0,3.0,0.0,4.0,0.0,2.0,2.0,0.0,...,0.0,4.0,2.0,4.0,1.0,1.0,19.0,9.0,2.0,0.0
2016-01-14,4.0,5.0,9.0,6.0,5.0,4.0,3.0,1.0,1.0,3.0,...,3.0,4.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0


In [ ]:
"""
def news_organizer(ticker_list, filename):
    data = pd.read_csv(path+filename)
    data['TimeOfArrival'] = [datetime.strptime(t, '%Y-%m-%d %H:%M:%S') for t in [i[:-4] for i in data['TimeOfArrival']]]
    data = data.set_index('TimeOfArrival')
    data['arr_time'] = data.index
    
    # news count at 5 min interval
    freq_news_collection = []
    exact_arrival_collection = []
    #arrTime_df = None
    for ticker in ticker_list:
        _ticker_ = data[data['Ticker'] == ticker]
        freq_news_collection.append(list(_ticker_.groupby(pd.TimeGrouper('5Min'))['Ticker'].count()))
        
        
        v = []
        for key, value in BAC.groupby(pd.TimeGrouper('5Min'))['arr_time']:
            if list(value) != []:
                v.append([list(value)])
        
        #pd.DataFrame(v).to_csv('results/news_arrival_time/'+ticker+filename[:-4]+'.csv')

    freq_news_df = pd.DataFrame(freq_news_collection).transpose()
    freq_news_df.columns = ticker_list
    #freq_news_df.index = _ticker_.groupby(pd.TimeGrouper('5Min'))['Ticker'].count().keys()
    freq_news_df.fillna(0, inplace=True)
    print(freq_news_df)
    #freq_news_df.to_csv('results/5min_news_count_'+filename[:-4]+'.csv')
    """